In [2]:
if 'google.colab' in str(get_ipython()):
  !pip install PyCO2SYS

In [3]:
from itertools import product
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import PyCO2SYS as csys

In [4]:
# CO2SYS parameter types
# https://pyco2sys.readthedocs.io
par_type = dict(alk=1, dic=2, pH=3, pCO2=4)

# state information
# conditions that are constant
salinity = 35.0 # psu
temperature = 12.0 # °C
phosphate = 0.1 # mmol/m^3
silicate = 0.5 # mmol/m^3

# conditions that vary
index_keys = ['alkalinity', 'pCO2']
alk_range = np.arange(1900.0, 3000.0, 100)
pco2_range = np.arange(280.0, 500.0, 10.)

# results to save in the table
result_keys = [
    'dic', 'CO2', 'HCO3', 'CO3', 'pH',
    'saturation_calcite', 'saturation_aragonite',
]

# precompute equilibrium constants (for performance)
result = csys.sys(
      temperature=temperature,
      salinity=salinity,
      )
equil_constants = {k: v for k, v in result.items() if k[:2] == 'k_'}

# compute the lookup table
rows = []
for alk, pco2 in product(alk_range, pco2_range):
  result = csys.sys(
      par1=pco2,
      par2=alk,
      par1_type=par_type['pCO2'],
      par2_type=par_type['alk'],
      temperature=temperature,
      salinity=salinity,
      total_silicate=silicate,
      total_phosphate=phosphate,
      **equil_constants,
    )

  rows.append({k: result[k] for k in result_keys+index_keys})

df = pd.DataFrame(rows).set_index(keys=index_keys)
df

dic        CO2         HCO3         CO3        pH  \
alkalinity pCO2                                                               
1900.0     280.0  1701.711697  11.470667  1559.001299  131.239731  8.104618   
           290.0  1707.203817  11.880334  1567.262737  128.060746  8.091674   
           300.0  1712.448075  12.290000  1575.121449  125.036626  8.079123   
           310.0  1717.462669  12.699667  1582.606983  122.156019  8.066941   
           320.0  1722.264013  13.109334  1589.745995  119.408684  8.055108   
...                       ...        ...          ...         ...       ...   
2900.0     450.0  2643.944664  18.435000  2427.519604  197.990060  8.090880   
           460.0  2648.660125  18.844667  2434.943111  194.872347  8.082661   
           470.0  2653.239049  19.254334  2442.130904  191.853811  8.074601   
           480.0  2657.687987  19.664000  2449.094307  188.929680  8.066694   
           490.0  2662.013070  20.073667  2455.843912  186.095491  8.058935   

                  saturation_calcite  saturation_aragonite  
alkalinity pCO2                                             
1900.0     280.0            3.125998              1.995394  
           290.0            3.050278              1.947060  
           300.0            2.978246              1.901081  
           310.0            2.909633              1.857284  
           320.0            2.844195              1.815513  
...                              ...                   ...  
2900.0     450.0            4.715924              3.010279  
           460.0            4.641663              2.962877  
           470.0            4.569764              2.916983  
           480.0            4.500115              2.872524  
           490.0            4.432607              2.829432  

[242 rows x 7 columns]